# Importing the Libraries

In [ ]:
pip install firebase-admin

In [ ]:
import base64
from Crypto import Random
from Crypto.PublicKey import RSA


import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

import ast

import pandas as pd

# Defining Decrypting Functions:

In [ ]:
def decrypt(private_key, en_mssg):
    """
    This function decrypts the encrypted message(en_mssg)
    using the private_key
    """
    return private_key.decrypt(base64.b64decode(en_mssg))

# Retrieving the Key and Encrypted Messages from Firebase:

In [ ]:
cred= credentials.Certificate('../input/firebase-key/secret-talkie-firebase-adminsdk-uq14f-0d6e7ac817.json')

firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://secret-talkie.firebaseio.com/'
})

In [ ]:
#Retrieveing data
ref= db.reference('1st Publish/Encrypted key and mssgs/key')
key_str= ref.get()



#Making a dictionary(messages) which will contain, mssg number: encrypted message
messages={}
ref= db.reference('1st Publish/Encrypted key and mssgs/')
for i in range(1, len(ref.get())):
    ref_in= db.reference(f'1st Publish/Encrypted key and mssgs/')
    messages[f"message{i}"]= list(list(ref_in.get().values())[i-1].values())[0] #Due to multiple push, firebase returns the pushed values in form of a dictionaly with unique keys.
    

In [ ]:
#Viewing the encrypted message dictionary
messages

# Decrypting the Numerical Encrypted key:


In [ ]:
#The following list is copied from the transmiter notebook
list_char=['H', 'M', 'J', 'G', '-', '>', 'B', 'b', 'x', '\n', 'k', '~', 'n', '_', '(', 'C', 'j', 'I', 'p', 'f', 'u', 'Q', '0', 'T', 'd', 'v', 'R', '.', 'X', '}', '&', '`', '%', '^', '|', 'A', '1', '=', ':', 'Y', 'F', '2', 'K', ' ', 'w', 'o', 'P', ';', 'Z', "'", 'E', 'c', '5', 'l', '#', '+', 'y', '{', ']', 's', 'q', '6', '"', 'r', ')', 'g', 't', 'V', 'N', 'W', '8', 'L', 'O', '@', 'D', '*', '7', 'U', '$', '9', 'z', 'i', '/', '[', ',', 'm', '<', '4', '?', 'S', 'h', '!', 'e', '3', 'a']
list_lab=[]
for i in range(len(list_char)):
    list_lab.append(i)
    
list_lab_char=[]
for i in range(0, len(list_char)):
    list_lab_char.append([list_lab[i], list_char[i]])

In [ ]:
#Converting String format to List
key_list= ast.literal_eval(key_str)


#getting the original key from the numeric encryption in list format
key_retrived=[]
for i in range(len(key_list)):
    for j in range(len(list_lab_char)):
        if key_list[i]==list_lab_char[j][0]:
            key_retrived.append(list_lab_char[j][1])
            
            
#Converting List format to String            
orignal_key= ''.join(str(e) for e in key_retrived)

In [ ]:
orignal_key

# Numeric decrypting of messages:

In [ ]:
def numeric_decrypting(key_str):   
    #Converting String format to List
    key_list= ast.literal_eval(key_str)


    #getting the original key from the numeric encryption in list format
    key_retrived=[]
    for i in range(len(key_list)):
        for j in range(len(list_lab_char)):
            if key_list[i]==list_lab_char[j][0]:
                key_retrived.append(list_lab_char[j][1])


    #Converting List format to String            
    decrypted= ''.join(str(e) for e in key_retrived)
    
    return(decrypted)

In [ ]:
num_decrypt_mssg_list=[]
for i in range(len(list(messages.values()))):
    num_decrypt_mssg_list.append(numeric_decrypting(list(messages.values())[i]))

In [ ]:
num_decrypt_mssg_list

# Decrypting the Encrypted messages:

In [ ]:
key= RSA.importKey(orignal_key)

In [ ]:
# Making a dictionary (decrypt_messages) which contain decrypted message number: decrypted message
decrypt_messages={}

for i in range(len(messages)):
    decrypt_mssg= decrypt(key,num_decrypt_mssg_list[i])
    decrypt_messages[f'decrypt_message{i+1}']= decrypt_mssg.decode() #Might give wrong output due to decoding errors 

'''
If not giving proper decoded output, try following:
# n= list(messages.values())[0]
# decrypted= decrypt(key,n)
# decrypted_out= decrypted.decode()
# print(decrypted_out)
# print(type(decrypted_out)) #should be str
'''

In [ ]:
#Viewing the decrypted message dictionary
decrypt_messages

# Estimating the message Emotion

In [ ]:
data= pd.read_csv('../input/emotion-sensor-datasetfull/andbrainDataSet108.csv')


#Data processing;

def remove(string): 
    '''
    This function removes spaces fromwords(if any)
    and converts all letters to lower case
    '''
    string= string.replace(" ", "")
    string= string.lower()
    return string

# Removing spaces(if any) and converting all letters of word to lower case
for i in range(len(data)):
    data["word"][i]= remove(data["word"][i])
    
display(data.head(5))

In [ ]:
def find_message_emotion(mssg):
    
    #Convert all upper case letters of string to lower case
    mssg_lower= mssg.lower()

    #Convert string into list of words
    mssg_list= mssg_lower.split( )
    
    # Getting list of position numbers of words contained by data and message 
    sentiment_words=[]

    for i in range(len(mssg_list)):
        for j in range(len(data)):
            if mssg_list[i]== data["word"][j]:
                sentiment_words.append(j)


    # Initializing variables for probablity of each class
    p_disgust= 0
    p_surprise= 0
    p_neutral= 0
    p_anger= 0
    p_sad= 0
    p_happy= 0
    p_fear= 0   
    # Calculating the total probablities of each class and storing in same variable
    for i in sentiment_words:
        p_disgust= p_disgust + data["disgust"][i]
        p_surprise= p_surprise + data["surprise"][i]
        p_neutral= p_neutral + data["neutral"][i]
        p_anger= p_anger + data["anger"][i]
        p_sad= p_sad + data["sad"][i]
        p_happy= p_sad + data["happy"][i]
        p_fear= p_fear + data["fear"][i]


    # Defining a dicionary with output statement for message corrosponding to respective probablity values    
    Probablities = {
        'Emotion is Neutral':p_neutral,
        'Emotion is Disgust':p_disgust,
        'Emotion is Surprise':p_surprise,
        'Emotion is Anger':p_anger,
        'Emotion is Sad':p_sad,
        'Emotion is Happy':p_happy,
        'Emotion is Fear':p_fear
    }
    return (max(Probablities, key=Probablities.get)) #Returns key with greatest value(probablity)

In [ ]:
#Making a dictionary(messages_emotions) that contain,  number of the decrypted message: Emotion of the message
messages_emotions={}
for i in range(len(decrypt_messages)):
    emotion_message= find_message_emotion(list(decrypt_messages.values())[i])
    messages_emotions[f"[Index {i+1}]  " + list(decrypt_messages.values())[i]]= emotion_message

In [ ]:
messages_emotions